In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [0]:
# This should be the only variable to change per corpus
NAME = 'kanye'

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

PATH = '/content/drive/My Drive/LA Hacks/' 
DICT_PATH = PATH + 'Dictionaries/' + 'char_index_' + NAME + '.p'
MODEL_PATH = PATH + 'Model Weights/' + NAME + '.h5'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# Load corpus
path = PATH + 'kanye_corpus.txt'
with open(path, encoding='utf-16') as f:
    text = f.read().lower()

In [5]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 68


In [0]:
import pickle

with open(DICT_PATH, 'wb') as f:
  pickle.dump(char_indices, f)

In [7]:
print(chars)

['\n', ' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '·', 'á', 'é', 'í', 'ñ', 'ó', 'ā', '\u200b', '–', '“', '”', '…']


In [8]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 86767


In [9]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [10]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
# Load model
model.load_weights(MODEL_PATH)

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

checkpoint = ModelCheckpoint(MODEL_PATH, monitor='loss', verbose=0, save_best_only=True, save_weights_only=True)

early_stopping = EarlyStopping(monitor='loss', patience=3)

generate_text = LambdaCallback(on_epoch_end=on_epoch_end)


In [0]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[generate_text, checkpoint, early_stopping])

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
86767/86767 [==============================] - 63s 727us/step - loss: 2.2467

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "k with you"
i done did shit on logical
m"
k with you"
i done did shit on logical
my be to me to the got a me to the pest in the got the got a be the wert the sting a with a for a the we like the be with i got a me a like to the chat in the got a don't i'm so the still i'm she we like the we to the mane the stire to mace to the we the we like the wer a still be the be the ling so the all the we the be and the ching and the pell i was the weel the got it the got be to the peal so
----- diversity: 0.5
----- Generating with seed: "k with you"
i done did shit on logical
m"
k with you"
i done did shit on logical
my pup and thing as to see her some i scint bet come and the stas for see her couse on my a beet reat the never my mall that see to stippored me with i tay the pe

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


r can't tell your life that we ain't talkin' bout the soul
not the way i ain't got a fresh and the she was all that nigga with the way i can't rapper the charu to my brand and the whole could we got a broke
i ain't been to the chird
they say they say they say not on th
----- diversity: 0.5
----- Generating with seed: "i'm tryin' to spend my stacks
and i'm so"
i'm tryin' to spend my stacks
and i'm so flace
when i go to the sumer can but they put to the way and you hot the facks
i got her the straight that niggas con at here
they have a past to do a benz
we never like you tell 'em have we streets and they think i don't get the facks
the some hop
you don't don't know i have a pay on your drunk to the whole got a baby
but i think i got a bolly ringins
and they don't know what the niggas
i'm talk
----- diversity: 1.0
----- Generating with seed: "i'm tryin' to spend my stacks
and i'm so"
i'm tryin' to spend my stacks
and i'm so around
aid a ur
fired again, "i niggar call, have it? rappers
th

In [0]:
!ls

drive  gdrive  nietzsche.h5  sample_data
